In [92]:
import glob
import random

import joblib
import numpy as np
import pandas as pd

from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [36]:
# df = pd.read_csv("../dataset/xAllChanges.csv")
df = pd.read_csv("../dataset/xNoChange.csv")

df.head()

,Unnamed: 0,Unnamed: 0.1,xSubscriptionId_fk,xCycleCode,xFamilyNum,xFaze,xAmper,xCounterBuldingNo,xRegionName_Roustaei,xRegionName_Shahri,xUsageGroupName_Keshavarzi,xUsageGroupName_Khanegi,xUsageGroupName_Omoomi,xUsageGroupName_Sanati,xUsageGroupName_Sayer,xBakhshCode_1,xBakhshCode_2,xBakhshCode_4,xTimeControlCode_1,xTimeControlCode_2,xTimeControlCode_3,xTariffOldCode_1010,xTariffOldCode_1011,xTariffOldCode_1110,xTariffOldCode_1111,xTariffOldCode_1990,xTariffOldCode_2110,xTariffOldCode_2210,xTariffOldCode_2310,xTariffOldCode_2410,xTariffOldCode_2510,xTariffOldCode_2610,xTariffOldCode_2710,xTariffOldCode_2990,xTariffOldCode_2992,xTariffOldCode_3110,xTariffOldCode_3210,xTariffOldCode_3310,xTariffOldCode_3410,xTariffOldCode_3520,xTariffOldCode_3540,xTariffOldCode_3740,xTariffOldCode_3991,xTariffOldCode_4410,xTariffOldCode_4610,xTariffOldCode_4990,xTariffOldCode_5110,xTariffOldCode_5990,days_difference,month,mediumDailyUsage,highDailyUsage,lowDailyUsage,xMeduimKw,xHighKw,xLowKw
0,0,0,9397665,25,1,3,25,12606909,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,8,38.428571,0.0,0.0,538,0,0
1,1,1,9396214,44,1,3,25,8336853341,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,8,95.190476,0.0,0.0,1999,0,0
2,2,2,9396214,44,1,3,25,8336853341,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,9,0.000000,0.0,0.0,0,0,0
3,3,3,8952093,32,1,3,25,37337459,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,11,15.661765,0.0,0.0,1065,0,0
4,4,4,8952093,32,1,3,25,37337459,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,62,2,9.032258,0.0,0.0,560,0,0


## Creating the Model

In [53]:
features = ['xCycleCode', 'xFamilyNum', 'xFaze', 'xAmper', 'xRegionName_Roustaei',
                'xRegionName_Shahri', 'xUsageGroupName_Keshavarzi', 'xUsageGroupName_Khanegi',
                'xUsageGroupName_Omoomi', 'xUsageGroupName_Sanati', 'xUsageGroupName_Sayer',
                'xBakhshCode_1', 'xBakhshCode_2', 'xBakhshCode_4',
                'xTimeControlCode_1', 'xTimeControlCode_2', 'xTimeControlCode_3',
                'xTariffOldCode_1010', 'xTariffOldCode_1011', 'xTariffOldCode_1110',
                'xTariffOldCode_1111', 'xTariffOldCode_1990', 'xTariffOldCode_2110',
                'xTariffOldCode_2210', 'xTariffOldCode_2310', 'xTariffOldCode_2410',
                'xTariffOldCode_2510', 'xTariffOldCode_2610', 'xTariffOldCode_2710',
                'xTariffOldCode_2990', 'xTariffOldCode_2992', 'xTariffOldCode_3110',
                'xTariffOldCode_3210', 'xTariffOldCode_3310', 'xTariffOldCode_3410', 
                'xTariffOldCode_3520', 'xTariffOldCode_3540', 'xTariffOldCode_3740', 
                'xTariffOldCode_3991', 'xTariffOldCode_4410', 'xTariffOldCode_4610', 
                'xTariffOldCode_4990', 'xTariffOldCode_5110', 'xTariffOldCode_5990',
                'days_difference', 'month']


X = df[features]
y_medium = df["mediumDailyUsage"]
y_high = df["highDailyUsage"]
y_low = df["lowDailyUsage"]

# X_train, X_test, y_train, y_test = train_test_split(X, y_medium)

In [6]:
rng = np.random.RandomState(1)
regr = AdaBoostRegressor(DecisionTreeRegressor(),
                          n_estimators=10 , random_state=rng)

regr.fit(X, y_low)
joblib.dump(regr, '../models/ABR-NoChanges-Low.pkl')

print('Model has benn saved to disk successfully.')

Model has benn saved to disk successfully.


In [24]:
# df = pd.DataFrame(df['xSubscriptionId_fk'].unique(),columns= ['xSubscriptionId_fk'])
# df.to_csv("../dataset/xNoChange_xSubscriptionId_fk.csv")

## Serving the Model

In [124]:
all_medium_model = joblib.load('../models/ABR-AllChanges-Medium.pkl')
all_high_model = joblib.load('../models/ABR-AllChanges-High.pkl')
all_low_model = joblib.load('../models/ABR-AllChanges-Low.pkl')

no_medium_model = joblib.load('../models/ABR-NoChange-Medium.pkl')
no_high_model = joblib.load('../models/ABR-NoChange-High.pkl')
no_low_model = joblib.load('../models/ABR-NoChange-Low.pkl')

In [125]:
all_df = pd.read_csv('../dataset/xAllChanges.csv')
no_df = pd.read_csv('../dataset/xNoChange.csv')

In [126]:
all_change_ids = pd.read_csv('../dataset/xAllChanges_xSubscriptionId_fk.csv')
no_change_ids = pd.read_csv('../dataset/xNoChange_xSubscriptionId_fk.csv')

In [347]:
def predict(user_id, days, month):
    """
    Get predictions for a particular user.
    """
    try:
        medium_predicted = 0
        high_predicted = 0
        low_predicted = 0

        message = None
                                                
        if len(all_change_ids.loc[all_change_ids['xSubscriptionId_fk'] == user_id]) > 0:
            df = all_df.loc[all_df['xSubscriptionId_fk'] == user_id]
            df = df.iloc[-1:]
            df = df.drop(['xSubscriptionId_fk'], 1)
            df = df.drop(['xCounterBuldingNo'], 1)
            df = df.drop(df.columns[[0, 1]], 1).iloc[:, : 48]

            df['days_difference'] = days
            df['month'] = month
            
            if df['xTimeControlCode_1'].iloc[0] == 1:
                medium_predicted = all_medium_model.predict(df[features])[0] * days
            if df['xTimeControlCode_2'].iloc[0] == 1:
                medium_predicted = all_medium_model.predict(df[features])[0] * days
                high_predicted = all_high_model.predict(df[features])[0] * days
            if df['xTimeControlCode_3'].iloc[0] == 1:
                low_predicted = all_low_model.predict(df[features])[0] * days
                medium_predicted = all_medium_model.predict(df[features])[0] * days
                high_predicted = all_high_model.predict(df[features])[0] * days

        else:
            df = no_df.loc[no_df['xSubscriptionId_fk'] == user_id]
            df = df.iloc[-1:]
            df = df.drop(['xSubscriptionId_fk'], 1)
            df = df.drop(['xCounterBuldingNo'], 1)
            df = df.drop(df.columns[[0, 1]], 1).iloc[:, : 48]

            df['days_difference'] = days
            df['month'] = month
            
            if df['xTimeControlCode_1'].iloc[0] == 1:
                medium_predicted = no_medium_model.predict(df[features])[0] * days
            if df['xTimeControlCode_2'].iloc[0] == 1:
                medium_predicted = no_medium_model.predict(df[features])[0] * days
                high_predicted = no_high_model.predict(df[features])[0] * days
            if df['xTimeControlCode_3'].iloc[0] == 1:
                low_predicted = no_low_model.predict(df[features])[0] * days
                medium_predicted = no_medium_model.predict(df[features])[0] * days
                high_predicted = no_high_model.predict(df[features])[0] * days

        return {
            "success": True, 
            "message": message, 
            "prediction": {
                "medium_predicted": round(medium_predicted, 10), 
                "high_predicted": round(high_predicted, 10), 
                "low_predicted": round(low_predicted, 10) }
        }

    except Exception as ex:
        error_content = {
            "success": False,
            "message": "Hello! Error Happened.",
            "code": "PREDICTION_ERROR",
            "properties": {
                "code": 1000,
                "message": ex.__str__(),
            },
        }
        return error_content

In [348]:
predict(110402, 60, 6)

{'message': None,
 'prediction': {'high_predicted': 67.0,
  'low_predicted': 133.3333333333,
  'medium_predicted': 277.0},
 'success': True}

## Testing the Model

In [267]:
def random_User_selector(samples=5, mode='no-change'):
    random_user_ids = []
    if mode == 'all-changes':
        random_users = random.sample(list(all_change_ids['xSubscriptionId_fk']), samples)
    else:
        random_users = random.sample(list(no_change_ids['xSubscriptionId_fk']), samples)
        
    return random_users

In [340]:
def test(cases):
    for random_user_id in random_user_ids:
        try:
            df = pd.read_csv("../dataset/users_data/{0}.csv".format(random_user_id))
            
            prediction = predict(random_user_id, df.iloc[-1:]["days_difference"].iloc[-1], df.iloc[-1:]["month"].iloc[-1])
                
            if prediction["success"]:
                print('User: {0} Expected and Predicted values for month: {1} and days: {2} are:'.format(random_user_id, df.iloc[-1:]["month"].iloc[-1], df.iloc[-1:]["days_difference"].iloc[-1]))
                if prediction["message"]:
                    pass
                    print(prediction["message"])

                print('    Medium: {0}----->{1}'.format( df.iloc[-1:]["xMeduimKw"].iloc[-1], prediction["prediction"]["medium_predicted"]))
                error_calc(df.iloc[-1:]["xMeduimKw"].iloc[-1], prediction["prediction"]["medium_predicted"])
                print('    High: {0}------->{1}'.format( df.iloc[-1:]["xHighKw"].iloc[-1], prediction["prediction"]["high_predicted"]))
                error_calc(df.iloc[-1:]["xHighKw"].iloc[-1], prediction["prediction"]["high_predicted"])
                print('    Low: {0}-------->{1}'.format( df.iloc[-1:]["xLowKw"].iloc[-1], prediction["prediction"]["low_predicted"]))
                error_calc(df.iloc[-1:]["xLowKw"].iloc[-1], prediction["prediction"]["low_predicted"])
        except Exception as excp:
            print("CSV not found. \n {0}".format(excp.__str__()))

### Calculate error for random users who had changes

In [344]:
random_user_ids = random_User_selector(samples=5, mode='all-changes')
test(random_user_ids)

User: 9624281 Expected and Predicted values for month: 1 and days: 10 are:
    Medium: 0----->2.0
    High: 0------->0.0
    Low: 0-------->2.0
User: 126339 Expected and Predicted values for month: 8 and days: 61 are:
    Medium: 581----->398.4444444444
    High: 0------->0
    Low: 0-------->0
User: 9189091 Expected and Predicted values for month: 5 and days: 30 are:
    Medium: 123----->110.5
    High: 65------->60.5
    Low: 104-------->92.0
User: 111531 Expected and Predicted values for month: 11 and days: 71 are:
    Medium: 5----->496.4
    High: 0------->0
    Low: 0-------->0
User: 2110030 Expected and Predicted values for month: 3 and days: 59 are:
    Medium: 250----->373.9285714286
    High: 123------->144.0909090909
    Low: 134-------->158.0666666667


### Calculate error for random users who did not have changes

In [364]:
random_user_ids = random_User_selector(samples=5, mode='no-change')
test(random_user_ids)

User: 9176287 Expected and Predicted values for month: 8 and days: 56 are:
    Medium: 113----->134.5428571429
    High: 45------->63.7377483444
    Low: 66-------->96.6742323097
User: 9495892 Expected and Predicted values for month: 12 and days: 64 are:
    Medium: 166----->145.2282157676
    High: 60------->65.2644628099
    Low: 94-------->89.9911111111
User: 119930 Expected and Predicted values for month: 7 and days: 57 are:
    Medium: 555----->366.9842519685
    High: 0------->0
    Low: 0-------->0
User: 9450852 Expected and Predicted values for month: 7 and days: 59 are:
    Medium: 196----->158.9230769231
    High: 78------->65.6779661017
    Low: 112-------->93.4393939394
User: 9016783 Expected and Predicted values for month: 12 and days: 65 are:
    Medium: 555----->356.1
    High: 0------->0
    Low: 0-------->0


## The Subscriptions with multiple values in xFaze

In [366]:
random_user_ids = [110003, 110019, 110037, 110059, 110063]
test(random_user_ids)

User: 110003 Expected and Predicted values for month: 8 and days: 59 are:
    Medium: 298----->176.5904761905
    High: 139------->91.2380952381
    Low: 179-------->127.71875
User: 110019 Expected and Predicted values for month: 6 and days: 1180 are:
    Medium: 1880----->1880.0
    High: 0------->0
    Low: 0-------->0
User: 110037 Expected and Predicted values for month: 1 and days: 52 are:
    Medium: 178----->380.868852459
    High: 0------->0
    Low: 0-------->0
User: 110059 Expected and Predicted values for month: 12 and days: 59 are:
    Medium: 348----->533.2
    High: 0------->0
    Low: 0-------->0
User: 110063 Expected and Predicted values for month: 4 and days: 58 are:
    Medium: 171----->169.8571428571
    High: 69------->64.2142857143
    Low: 87-------->85.375


## The Subscriptions with multiple values in xAmper

In [367]:
random_user_ids = [110003, 110059, 110078, 110079, 110082]
test(random_user_ids)

User: 110003 Expected and Predicted values for month: 8 and days: 59 are:
    Medium: 298----->176.5904761905
    High: 139------->91.2380952381
    Low: 179-------->127.71875
User: 110059 Expected and Predicted values for month: 12 and days: 59 are:
    Medium: 348----->533.2
    High: 0------->0
    Low: 0-------->0
User: 110078 Expected and Predicted values for month: 2 and days: 57 are:
    Medium: 18----->185.0909090909
    High: 14------->61.0625
    Low: 15-------->97.5882352941
User: 110079 Expected and Predicted values for month: 9 and days: 24 are:
    Medium: 6----->6.0
    High: 0------->0.0
    Low: 0-------->0.0
User: 110082 Expected and Predicted values for month: 8 and days: 58 are:
    Medium: 294----->382.125
    High: 0------->0
    Low: 0-------->0


## The Subscriptions with multiple values in xCounterBuldingNo

In [368]:
random_user_ids = [110003, 110005, 110006, 110007, 110010]
test(random_user_ids)

User: 110003 Expected and Predicted values for month: 8 and days: 59 are:
    Medium: 298----->176.5904761905
    High: 139------->91.2380952381
    Low: 179-------->127.71875
User: 110005 Expected and Predicted values for month: 6 and days: 62 are:
    Medium: 511----->455.8709677419
    High: 0------->0
    Low: 0-------->0
User: 110006 Expected and Predicted values for month: 2 and days: 118 are:
    Medium: 0----->906.0
    High: 0------->0
    Low: 0-------->0
User: 110007 Expected and Predicted values for month: 6 and days: 60 are:
    Medium: 221----->188.09375
    High: 86------->91.768115942
    Low: 134-------->138.3103448276
User: 110010 Expected and Predicted values for month: 6 and days: 56 are:
    Medium: 90----->189.6724137931
    High: 49------->82.0526315789
    Low: 76-------->131.7868852459
